## Import dataset 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re

In [2]:
plt.style.use('ggplot')
%matplotlib inline

In [3]:
df_all = pd.read_csv('data/april_report.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_all.columns = [col.lower().replace(' ','_') for col in df_all.columns.tolist()]
df_all.columns = [col.lower().replace('/timestamp','') for col in df_all.columns.tolist()]

In [5]:
date_col = [col for col in df_all.columns.tolist() if 'date' in col]
df_all[date_col] = df_all[date_col].apply(pd.to_datetime,format = "%m/%d/%Y %H:%M:%S")

In [6]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253306 entries, 0 to 1253305
Data columns (total 23 columns):
unique_row_id           1253306 non-null object
cvo_sid                 1253306 non-null object
event_site_id           1253306 non-null int64
touchpoint_site_id      1253306 non-null int64
event_site_name         1253306 non-null object
touchpoint_site_name    1253306 non-null object
event_type              1253306 non-null object
event_id                1253306 non-null object
event_date              1253306 non-null datetime64[ns]
touchpoint_date         1253306 non-null datetime64[ns]
total_event_value       1253306 non-null float64
touchpoint_revenue      1253306 non-null float64
conversion              1253306 non-null float64
touchpoint_type         1027589 non-null object
first_touch_date        1253306 non-null datetime64[ns]
last_touch_date         1253306 non-null datetime64[ns]
source_1                1253306 non-null object
source_2                1253306 non-nu

## Clean
* pick columns
* remove channel base


In [19]:
selected_col=['event_site_name','touchpoint_site_name','event_type','conversion','touchpoint_type','source_1','source_3','touchpoints_in_path','attribution_sequence','path_position']
selected_col.extend(date_col)

df_prep=df_all[selected_col]

In [20]:
df_prep = df_prep[df_prep.source_1!='base']
# df_prep = df_prep[df_prep.touchpoints_in_path<50]
df_prep.reset_index(drop=True, inplace=True)
df_prep.loc[df_prep.source_1.str.contains('social'),'source_1']='social'

In [21]:
def remove_small_category(df,col):
    count = df[col].value_counts()
    index = count[count<1000].index.tolist()
    return df[~df[col].isin(index)]

In [22]:
df_prep = remove_small_category(df_prep,'source_1')
df_prep = remove_small_category(df_prep,'event_type')

In [24]:
df_prep.source_1.value_counts()

display                 726559
video                   164456
ppc brand                36791
organic not provided     22399
content                  18998
other                    15348
social                   14918
coupon                   11951
ppc nonbrand              7640
email                     6023
organic brand             1452
Name: source_1, dtype: int64

In [25]:
df_prep.shape

(1026535, 14)

## procesing
* conbime servel social
* days from first to last touch
* distance to event
* mean of __event, source, path distant, path position, touch point type__

In [26]:
df_prep['event_to_touchpoint']=(df_prep.event_date-df_prep.touchpoint_date).dt.days

df_prep['day_of_week']=df_prep.touchpoint_date.dt.weekday

df_prep['month']=df_prep.touchpoint_date.dt.month

df_prep['week_of_month']=(df_prep.touchpoint_date.dt.day-1)//7+1

df_prep['distance_event'] = df_prep.touchpoints_in_path - df_prep.attribution_sequence

In [28]:
df_prep.head(20)

,event_site_name,touchpoint_site_name,event_type,conversion,touchpoint_type,source_1,source_3,touchpoints_in_path,attribution_sequence,path_position,event_date,touchpoint_date,first_touch_date,last_touch_date,event_to_touchpoint,day_of_week,month,week_of_month,distance_event
0,Nexium24hour,Nexium24hour,nexium24hour-buy.now,1.000000,visit,ppc nonbrand,+cure +heartburn,1,1,only,2017-04-07 17:54:58,2017-04-07 17:54:58,2017-04-07 17:54:58,2017-04-07 17:54:58,0,4,4,1,0
1,Centrum,Centrum,centrum-coupon.submit,0.000047,visit,organic not provided,centrum.com/offers,2,1,introducer,2017-04-21 17:52:51,2017-04-12 12:26:32,2017-04-12 12:26:32,2017-04-21 17:52:51,9,2,4,2,1
2,Centrum,Centrum,centrum-coupon.submit,0.011891,visit,organic not provided,centrum.com/offers,2,2,closer,2017-04-21 17:52:51,2017-04-21 17:50:56,2017-04-12 12:26:32,2017-04-21 17:52:51,0,4,4,3,0
3,Centrum,Centrum,centrum-coupon.submit,0.197314,visit,ppc brand,centrum coupon,1,1,only,2017-04-30 18:04:52,2017-04-30 18:04:22,2017-04-30 18:04:22,2017-04-30 18:04:52,0,6,4,5,0
4,Centrum,Centrum,centrum-email.sign.up,1.000000,visit,social,rffb_dlxmultibuyersorllbuyers_movingdadgif041717,1,1,only,2017-04-19 14:12:59,2017-04-19 14:12:00,2017-04-19 14:12:00,2017-04-19 14:12:00,0,2,4,3,0
5,Advil PM,Nexium24hour,sale.advilpm,0.000002,impression,content,142293644,4,1,introducer,2017-04-19 14:49:00,2016-12-21 11:39:44,2016-12-21 11:39:44,2017-04-19 14:49:00,119,2,12,3,3
6,Advil PM,Nexium24hour,sale.advilpm,0.000002,impression,content,141628154,4,2,influencer,2017-04-19 14:49:00,2016-12-22 23:41:43,2016-12-21 11:39:44,2017-04-19 14:49:00,117,3,12,4,2
7,Advil PM,Nexium24hour,sale.advilpm,0.000002,impression,content,142293645,4,3,influencer,2017-04-19 14:49:00,2016-12-25 08:50:56,2016-12-21 11:39:44,2017-04-19 14:49:00,115,6,12,4,1
8,Advil PM,Centrum,sale.advilpm,0.000002,impression,display,142449245,4,4,closer,2017-04-19 14:49:00,2017-04-14 18:37:58,2016-12-21 11:39:44,2017-04-19 14:49:00,4,4,4,2,0
9,Advil,Advil,sale.advil,0.014566,impression,video,143085908,1,1,only,2017-04-17 12:54:00,2017-02-08 07:57:06,2017-02-08 07:57:06,2017-04-17 12:54:00,68,2,2,2,0


In [15]:
df_prep['round_conversion'] = df_prep.conversion.round(decimals=4)

In [72]:
df_prep['cate'] = pd.cut(df_prep.conversion,100,labels=False)

 Get Dummies

In [74]:
df_prep_dummies=pd.get_dummies(df_prep,columns=['event_site_name','event_type','touchpoint_type','source_1','path_position'],
                          drop_first=True)

In [75]:
#drop column with date
df_model=df_prep_dummies.drop(date_col,axis=1)

In [18]:
from sklearn.model_selection import train_test_split

In [87]:
label = df_model.cate
feature = df_model.drop(['conversion','round_conversion','cate'],axis=1)

In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
label = le.fit_transform(label)

In [88]:
df_train,df_test,label_train,label_test = train_test_split(feature,label,test_size=.3,random_state=0)

## modeling

In [104]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

In [78]:
from sklearn.naive_bayes import GaussianNB

In [89]:
clf = GaussianNB()
clf.fit(df_train,label_train)

GaussianNB(priors=None)

In [90]:
pred = clf.predict(df_test)

In [91]:
metrics.accuracy_score(label_test,pred)

0.72503336461435053

In [44]:
a1 = le.inverse_transform(pred)

In [92]:
pred[:100]

array([ 0,  0,  0,  0,  3,  0,  0, 11,  0,  0,  0,  4,  0, 33,  0,  0, 79,
        0,  1,  0,  0,  0,  0,  1,  0,  0, 55, 55,  0,  0,  0,  1,  0,  0,
        0, 33,  0,  0,  0,  0,  1,  0, 44, 77,  0,  0, 94,  0,  0,  0,  1,
       25,  0,  0, 33,  0,  0,  0, 45,  0,  0,  0,  0, 73,  0,  0,  0,  0,
        0, 33,  0,  0, 79,  0, 76,  0,  0,  0, 24,  0,  5,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, 45,  5, 33,  0,  0, 33], dtype=int64)

In [83]:
label_test[:100].values

array([ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  6,
        0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, 26,  0,  0, 50,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  5,  0,  0,  0,  0,
        0,  2,  0,  0, 10,  0, 99,  0,  0,  0,  2,  1,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, 99,  0,  0,  0,  0,  0], dtype=int64)

In [ ]:
clf = svm.SVC()
clf.fit(df_train,label_train,)

In [43]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import (Ridge, TheilSenRegressor, RANSACRegressor, HuberRegressor)

from sklearn.ensemble import (RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor,
                              AdaBoostRegressor, GradientBoostingRegressor)

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn import metrics

* random forest

In [ ]:
rf = RandomForestRegressor(n_estimators=50,max_depth=10,random_state=0)

In [ ]:
rf.fit(df_train,label_train)

In [ ]:
rf.score(df_test,label_test)

In [ ]:
predict1 = rf.predict(df_test)

metrics.mean_squared_error(label_test,predict1)

* knn

In [ ]:
knn = KNeighborsRegressor(n_neighbors=10)

In [ ]:
knn.fit(df_train,label_train)

In [ ]:
knn.score(df_test,label_test)

* elstic net

In [ ]:
eln = ElasticNet(alpha=0.05)

In [ ]:
eln.fit(df_train,label_train)

In [ ]:
eln.score(df_test,label_test)

* grainting boosting

In [ ]:
gbr = GradientBoostingRegressor(random_state=0)

In [ ]:
gbr.fit(df_train,label_train)

In [ ]:
gbr.score(df_test,label_test)

* bagging regressor

In [ ]:
bag = BaggingRegressor(random_state=0)

In [ ]:
bag.fit(df_train,label_train)

In [ ]:
bag.score(df_test,label_test)

In [ ]:
predict1 = bag.predict(df_test)

metrics.mean_squared_error(label_test,predict1)